In [20]:
import numpy as np
import scipy.stats as scp 

We define $$X_T = (X_T^{(1)}, \dots , X_T^{(N)})$$

We have
$$\left\lbrace\begin{array}{l}
X_{\tau_j} = X_{\tau_{j-1}} + (r-\frac{V_{\tau_{j-1}}}{2}) \Delta t + \sqrt{V_{\tau_{j-1}}^+} \Delta W_{j} \\[.1cm]
V_{\tau_j} = V_{\tau_{j-1}} + \kappa (\theta - V_{\tau_{j-1}}) \Delta t + \sigma \sqrt{V_{\tau_{j-1}}^+} (\rho \Delta W_{j} + \sqrt{1-\rho^2} \Delta W_{j} ^ {\perp})
\end{array}\right.$$

with $\Delta W_{j} \sim N(0,\Delta t)$

In [23]:
def Heston_discret1(kappa_,theta_,sigma_,rho_,r_,T_,L_,V0_,S0_):
    V = [V0_]
    X = [np.log(S0_)]
    delta_t_ = T_/L_
    delta_W_ = np.array([scp.norm.rvs(loc = 0, scale = np.sqrt(delta_t_), size = L_)]).T
    delta_W_orth_ = np.array([scp.norm.rvs(loc = 0, scale = np.sqrt(delta_t_), size = L_)]).T
    for i in range(L_):
        Vi_ = V[i]+kappa_*(theta_-V[i])*delta_t_ + sigma_*np.sqrt(max(V[i],0))*(rho_*delta_W_[i][0] + np.sqrt(1-rho_**2)*delta_W_orth_[i][0])
        V.append(Vi_)
        Xi_ = X[i] + (r_-1/2*V[i])*delta_t_ + np.sqrt(max(V[i],0)) * delta_W_[i][0]
        X.append(Xi_)
    return X[L_]

In [24]:
X = Heston_discret1(2,0.04,0.5,-0.7,0.03,0.5,100,0.04,100)
X

4.7594720065896094

We define $$f(X_T) = (e^{X_T}-K)^+$$

In [25]:
def f(X_,K_):
    return max(np.exp(X_)-K_,0)

In [26]:
f_ = f(X,90)
f_

26.68430108960358

Monte Carlo :
$$ \mathbb{E}[f(X_T)] = \frac{1}{N} \sum_{i=1}^N f(X_T^{(i)})$$

In [27]:
def monte_carlo(kappa_,theta_,sigma_,rho_,r_,T_,L_,V0_,S0_,K0_,N_):
    X = []
    for i in range(N_):
        X.append(Heston_discret1(kappa_,theta_,sigma_,rho_,r_,T_,L_,V0_,S0_))
    s = 0
    for i in range(N_):
        s += f(X[i],K0_)
    return s/N_

In [28]:
MC = monte_carlo(2,0.04,0.5,-0.7,0.03,0.5,100,0.04,100,90,10000)
MC

13.395895470217912

We get 
$$ C \approx \frac{e^{-rT}}{N}\sum_{i=1}^N f(X_T^{(i)})$$

In [29]:
def call(esp_,r_,T_):
    return np.exp(-r_*T_)*esp_

In [30]:
call(MC,0.03,0.5)

13.196456569386248